In [60]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
import numpy as np
from fractions import Fraction

In [61]:
def mod(a, power):
    U = QuantumCircuit(16)        
    U.x(15)
    U.x(0)
    U.x(2)
    U.x(3)
    U = U.to_gate()
    c_U = U.control()
    return c_U

In [62]:
def qft_dagger(n):
    
    qc = QuantumCircuit(n)
    
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    
    return qc

In [63]:
N = 1343
a = 13

In [64]:
def qpe(a):
    n_count = 8
    qc = QuantumCircuit(16+n_count, n_count)
    
    
    for q in range(n_count):
        qc.h(q)     
    qc.x(15+n_count) 
    
    for q in range(n_count): 
        qc.append(mod(a, 2**q), 
                 [q] + [i+n_count for i in range(16)])
    
    qc.append(qft_dagger(n_count), range(n_count)) 
    qc.measure(range(n_count), range(n_count))
    
    return qc

In [65]:
qc = qpe(a) 

In [66]:
qasm_sim = Aer.get_backend('qasm_simulator')
t_qc = transpile(qc, qasm_sim)
obj = assemble(t_qc, shots=1)
result = qasm_sim.run(obj, memory=True).result()
readings = result.get_memory()
phase = int(readings[0],2)/(2**8)

In [67]:
frac = Fraction(phase).limit_denominator(1343)
s, r = frac.numerator, frac.denominator
print(r)

256


In [68]:
guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
print(guesses)

[17, 1]


In [69]:
first_factor = guesses[0]

In [70]:
print(first_factor)

17


In [71]:
second_factor = int(N/first_factor)

In [72]:
print(second_factor)

79


In [73]:
def modInverse(a, m):
    m0 = m
    y = 0
    x = 1
    if m == 1:
        return 0
    while a > 1:
        # q is quotient
        q = a // m
        t = m
        # m is remainder now, process
        # same as Euclid's algo
        m = a % m
        a = t
        t = y
        # Update x and y
        y = x - q * y
        x = t
    # Make x positive
    if x < 0:
        x = x + m0
    return x

In [74]:
n = 1343
#factors of n from shor
p = first_factor
q = second_factor

tf = (p-1) * (q-1)
e = 5 

enc = 212

In [75]:
#the private key
d = modInverse(e, tf)

#decrypted msg
dec = (enc**d)%n

print(enc)
print(dec)

212
1233


In [43]:
#decoded message = 1233 
#result = pass